In [111]:
import bokeh.io as io
import bokeh.models as models
import bokeh.plotting as plotting
import bokeh.resources as resources
import pandas as pd
from bokeh.models import ColumnDataSource
import math
from bokeh.io import output_notebook
import bokeh.palettes
import pickle
import numpy as np
import pandas as pd
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap, dodge
import math
from bokeh.layouts import column, row, grid, gridplot

In [112]:
import viewer.app.data as data

In [113]:
from bokeh.io import output_notebook

In [114]:
output_notebook()

Loading BokehJS ...

In [115]:
measures = data.get_data()[0]
matches = data.get_data()[1]

In [116]:
taxi = measures[measures['task'].isin(['taxi'])]
taxi

,match,measure1,measure2,measure_type,phase,task,team_number
1022,qm9,true,-1,boolean,auto,taxi,frc488
1024,qm9,true,-1,boolean,auto,taxi,frc4512
1028,qm9,true,-1,boolean,auto,taxi,frc4682
1029,qm9,true,-1,boolean,auto,taxi,frc5683
105,qm1,false,-1,boolean,auto,taxi,frc4915
...,...,...,...,...,...,...,...
914,qm8,true,-1,boolean,auto,taxi,frc949
944,qm8,false,-1,boolean,auto,taxi,frc4469
960,qm8,false,-1,boolean,auto,taxi,frc2929
978,qm8,false,-1,boolean,auto,taxi,frc4915


In [117]:
measures = measures[measures['team_number'].str.contains('frc')].copy()

In [118]:
measures['measure1'] = measures['measure1'].replace(['true'],2)
measures['measure1'] = measures['measure1'].replace(['false'],0)

In [119]:
measures[measures.task == 'taxi']

,match,measure1,measure2,measure_type,phase,task,team_number
1022,qm9,2,-1,boolean,auto,taxi,frc488
1024,qm9,2,-1,boolean,auto,taxi,frc4512
1028,qm9,2,-1,boolean,auto,taxi,frc4682
1029,qm9,2,-1,boolean,auto,taxi,frc5683
105,qm1,0,-1,boolean,auto,taxi,frc4915
...,...,...,...,...,...,...,...
914,qm8,2,-1,boolean,auto,taxi,frc949
944,qm8,0,-1,boolean,auto,taxi,frc4469
960,qm8,0,-1,boolean,auto,taxi,frc2929
978,qm8,0,-1,boolean,auto,taxi,frc4915


In [120]:
teams = list(measures['team_number'].unique())

In [121]:
m = measures[measures.team_number == team].copy()
m['phase_task'] = m['phase'] + "_" + m['task']
m[m.phase_task == 'tele_upper']

,match,measure1,measure2,measure_type,phase,task,team_number,phase_task
1914,qm16,5,3,count,tele,upper,frc2412,tele_upper
2285,qm19,0,11,count,tele,upper,frc2412,tele_upper
2915,qm24,4,6,count,tele,upper,frc2412,tele_upper
3770,qm30,0,0,count,tele,upper,frc2412,tele_upper
428,qm4,2,0,count,tele,upper,frc2412,tele_upper
4283,qm35,5,3,count,tele,upper,frc2412,tele_upper
5042,qm41,0,2,count,tele,upper,frc2412,tele_upper
5487,qm44,0,0,count,tele,upper,frc2412,tele_upper
5785,qm48,0,4,count,tele,upper,frc2412,tele_upper
6896,qm56,0,0,count,tele,upper,frc2412,tele_upper


In [122]:
allteams = []
for team in teams:
    m = measures[measures.team_number == team].copy()
    m['phase_task'] = m['phase'] + "_" + m['task']
    m = m[['match', 'team_number', 'phase_task', 'measure1']]
    filter1 = m['phase_task'].isin(['auto_upper', 'auto_lower', 'tele_upper', 'tele_lower', 'endgame_hangar_level_end', 'auto_taxi'])
    m = m[filter1]

    m['measure1'] = m['measure1'].astype('int32')

    tele_up, tele_low, auto_up, auto_low, auto_taxi = 0, 0, 0, 0, 0
    if('tele_upper' in m['phase_task'].unique()):
        tele_up = round(m[m['phase_task'] == 'tele_upper']['measure1'].sum() * 2/(len(m[m['phase_task'] == 'tele_upper']['measure1'])), 1)
    if('tele_lower' in m['phase_task'].unique()):
        tele_low = round(m[m['phase_task'] == 'tele_lower']['measure1'].sum()/(len(m[m['phase_task'] == 'tele_lower']['measure1'])), 1)
    if('auto_upper' in m['phase_task'].unique()):
        auto_up = round(m[m['phase_task'] == 'auto_upper']['measure1'].sum() * 4/(len(m[m['phase_task'] == 'auto_upper']['measure1'])), 1)
    if('auto_lower' in m['phase_task'].unique()):
        auto_low = round(m[m['phase_task'] == 'auto_lower']['measure1'].sum() * 2/(len(m[m['phase_task'] == 'auto_lower']['measure1'])), 1)
    if('endgame_hangar_level_end' in m['phase_task'].unique()):
        level = list(m[m['phase_task'] == 'endgame_hangar_level_end']['measure1'])
        sum = 0
        for i in level:
            if (i == 1):
                i = 4
            if (i == 2):
                i = 6
            if (i == 3):
                i = 10
            if (i == 4):
                i = 15
            sum += i
        hangar = round(sum / len(m[m['phase_task'] == 'endgame_hangar_level_end']['measure1']), 1)
    if('auto_taxi' in m['phase_task'].unique()):
        taxi = round(m[m['phase_task'] == 'auto_taxi']['measure1'].sum()/(len(m[m['phase_task'] == 'auto_taxi']['measure1'])), 1)
    pointslist = [team, tele_up, tele_low, auto_up, auto_low, hangar, taxi]
    allteams.append(pointslist)
df = pd.DataFrame(allteams, columns = ['team', 'tele_up', 'tele_low', 'auto_up', 'auto_low', 'hangar', 'taxi'])
df

,team,tele_up,tele_low,auto_up,auto_low,hangar,taxi
0,frc1294,4.2,0.0,2.9,0.0,5.2,1.5
1,frc4512,6.9,0.7,3.7,0.4,2.7,1.2
2,frc4915,8.7,0.2,1.7,0.0,0.0,1.5
3,frc949,0.0,1.5,0.0,1.2,0.0,2.0
4,frc4911,23.5,0.1,9.8,0.0,6.9,2.0
5,frc5683,0.0,4.3,0.7,1.2,3.8,1.7
6,frc488,0.2,5.2,0.0,2.3,14.6,2.0
7,frc7627,0.0,4.8,0.0,1.2,0.0,1.5
8,frc4682,0.0,0.3,1.0,0.2,2.0,1.7
9,frc948,0.0,5.5,0.0,0.8,2.0,1.7


In [123]:
df['total_points'] = df["tele_up"] + df["tele_low"] + df["auto_up"] + df["auto_low"] + df['hangar'] + df['taxi']
df.copy()

,team,tele_up,tele_low,auto_up,auto_low,hangar,taxi,total_points
0,frc1294,4.2,0.0,2.9,0.0,5.2,1.5,13.8
1,frc4512,6.9,0.7,3.7,0.4,2.7,1.2,15.6
2,frc4915,8.7,0.2,1.7,0.0,0.0,1.5,12.1
3,frc949,0.0,1.5,0.0,1.2,0.0,2.0,4.7
4,frc4911,23.5,0.1,9.8,0.0,6.9,2.0,42.3
5,frc5683,0.0,4.3,0.7,1.2,3.8,1.7,11.7
6,frc488,0.2,5.2,0.0,2.3,14.6,2.0,24.3
7,frc7627,0.0,4.8,0.0,1.2,0.0,1.5,7.5
8,frc4682,0.0,0.3,1.0,0.2,2.0,1.7,5.2
9,frc948,0.0,5.5,0.0,0.8,2.0,1.7,10.0


In [124]:
for i in range(0, len(df['team'])):
    df['team'].loc[i] = int(df['team'].iloc[i][3:])

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [125]:
df.sort_values(by = ['team'], inplace = True)
df['team'] = df['team'].astype(str)

In [126]:
filtered = ColumnDataSource(df)
plot = plotting.figure(x_range = filtered.data["team"],
            x_axis_label="Team",
            y_axis_label="Average Points",
            title= "Points",
            tools = "hover", 
            tooltips = "@team $name: @$name",
            width=1200, height=500)
tasks = ['tele_up', 'tele_low', 'auto_up', 'auto_low', 'hangar', 'taxi']

vbar = plot.vbar_stack(tasks,
                        x = "team", 
                        color = bokeh.palettes.Paired[len(tasks)], 
                        width = 0.9,
                        source = filtered)

legend_items = [(lbl, [glyph]) for lbl, glyph in zip(tasks, vbar)]
legend = models.Legend(items = legend_items, location = "top_right") 
plot.add_layout(legend, 'right')

plot.xaxis.major_label_orientation = math.pi/4
plotting.show(plot)

In [130]:
list(df.columns)

['team',
 'tele_up',
 'tele_low',
 'auto_up',
 'auto_low',
 'hangar',
 'taxi',
 'total_points']

In [133]:
df['total_points'] = round(df['total_points'],1)
df

,team,tele_up,tele_low,auto_up,auto_low,hangar,taxi,total_points
6,488,0.2,5.2,0.0,2.3,14.6,2.0,24.3
13,492,13.8,0.1,5.1,0.4,6.9,1.3,27.6
9,948,0.0,5.5,0.0,0.8,2.0,1.7,10.0
3,949,0.0,1.5,0.0,1.2,0.0,2.0,4.7
0,1294,4.2,0.0,2.9,0.0,5.2,1.5,13.8
18,1318,9.1,0.5,1.5,0.0,1.6,1.3,14.0
19,1778,10.0,1.1,2.0,0.0,3.2,1.3,17.6
22,1899,17.5,0.0,5.3,0.0,7.2,1.8,31.8
29,1983,7.5,0.3,2.9,0.0,0.0,1.8,12.5
30,2412,3.5,1.3,1.3,0.5,3.7,1.5,11.8


In [134]:
from bokeh.models import (Button, ColumnDataSource, CustomJS, DataTable,
                          NumberFormatter, RangeSlider, TableColumn)
source = ColumnDataSource(df)
columns = []
tasks = ['team', 'total_points','tele_up', 'tele_low', 'auto_up', 'auto_low', 'hangar', 'taxi']
for i in range(0, len(tasks)):
    columns.append(TableColumn(field = tasks[i], title = tasks[i].replace("_", " ")))
totalstb = DataTable(source = source, columns = columns)

In [129]:
show(totalstb)